## 1.1 使用jinja2模板
Jinja2 是一个 Python 的功能齐全的模板引擎。它有完整的 unicode 支持，一个可选 的集成沙箱执行环境，被广泛使用，以 BSD 许可证授权。

http://docs.jinkan.org/docs/jinja2/


**通过配置模板，可以把模板文件和数据文件进行分离。（模板文件）+ （数据）生成最终的设备配置。**

数据文件可以是相同的数据结构，这些数据的结构，可以使用yang module 进行定义。定义的yang module 用于数据的格式的校验。





In [7]:
from jinja2 import Environment, FileSystemLoader

In [8]:
# 加载模板的目录。在这个目录中包含了很多的配置模板文件。
env = Environment(loader=FileSystemLoader('./config_template/', 'utf-8'))

In [9]:
print(env.list_templates())

['.ipynb_checkpoints/vios_mgt-checkpoint.j2', '.ipynb_checkpoints/vios_ssh-checkpoint.j2', '.ipynb_checkpoints/xrv_hostname-checkpoint.j2', '.ipynb_checkpoints/xrv_interfaces-checkpoint.j2', '.ipynb_checkpoints/xrv_isis-checkpoint.j2', '.ipynb_checkpoints/xrv_lldp-checkpoint.j2', '.ipynb_checkpoints/xrv_mgt-checkpoint.j2', 'vios_interace.j2', 'vios_lldp.j2', 'vios_mgt.j2', 'vios_ssh.j2', 'xrv_bgp.j2', 'xrv_hostname.j2', 'xrv_interfaces.j2', 'xrv_isis.j2', 'xrv_lldp.j2', 'xrv_mgt.j2', 'xrv_sr.j2', 'xrv_ssh.j2']


In [10]:
# 加载一个模板文件，
hostname = env.get_template("xrv_hostname.j2")

In [11]:
#  使用变量渲染模板
print(hostname.render(hostname="xrv1",commit=True))


hostname xrv1


commit


end




## 1.2 从yaml文件中读取数据

In [12]:
import yaml

In [13]:
interfaces = yaml.load(open("./data/interfaces/xrv1_interfaces.yaml").read())

In [14]:
interfaces

{'interfaces': [{'ipaddress': '99.1.2.1', 'name': 'Gi0/0/0/0'},
  {'ipaddress': '99.1.2.5', 'name': 'Gi0/0/0/1'},
  {'ipaddress': '99.1.2.9', 'name': 'Gi0/0/0/2'},
  {'ipaddress': '99.1.2.13', 'name': 'Gi0/0/0/3'},
  {'ipaddress': '99.1.2.17', 'name': 'Gi0/0/0/4'},
  {'ipaddress': '99.1.2.21', 'name': 'Gi0/0/0/5'},
  {'ipaddress': '99.1.2.25', 'name': 'Gi0/0/0/6'},
  {'ipaddress': '99.1.0.1', 'name': 'Loopback0'}]}

In [15]:
interfaces_temp = env.get_template("xrv_interfaces.j2")

In [21]:
interfaces_cfgs = interfaces_temp.render(interfaces,commit=True)
print(interfaces_cfgs)



interface Gi0/0/0/0
  ipv4 address 99.1.2.1/30
  no shutdown

interface Gi0/0/0/1
  ipv4 address 99.1.2.5/30
  no shutdown

interface Gi0/0/0/2
  ipv4 address 99.1.2.9/30
  no shutdown

interface Gi0/0/0/3
  ipv4 address 99.1.2.13/30
  no shutdown

interface Gi0/0/0/4
  ipv4 address 99.1.2.17/30
  no shutdown

interface Gi0/0/0/5
  ipv4 address 99.1.2.21/30
  no shutdown

interface Gi0/0/0/6
  ipv4 address 99.1.2.25/30
  no shutdown

interface Loopback0
  ipv4 address 99.1.0.1/30
  no shutdown



commit


end




## 1.3 下发配置到设备上

In [20]:
from netmiko import ConnectHandler
iosxr1 = {
    'device_type': 'cisco_xr',
    'ip':   '172.20.3.101',      #  登录设备的IP地址
    'username': 'admin',
    'password': 'admin',
    'port' : 22                  # optional, defaults to 22    
}

net_connect = ConnectHandler(**iosxr1)

In [22]:
net_connect.send_config_set(interfaces_cfgs)

'config term\n\nFri Aug 10 03:02:01.548 UTC\nRP/0/0/CPU0:xrv1(config)#\n\nRP/0/0/CPU0:xrv1(config)#\n\nRP/0/0/CPU0:xrv1(config)#interface Gi0/0/0/0\n\nRP/0/0/CPU0:xrv1(config-if)#  ipv4 address 99.1.2.1/30\n\nRP/0/0/CPU0:xrv1(config-if)#  no shutdown\n\nRP/0/0/CPU0:xrv1(config-if)#\n\nRP/0/0/CPU0:xrv1(config-if)#interface Gi0/0/0/1\n\nRP/0/0/CPU0:xrv1(config-if)#  ipv4 address 99.1.2.5/30\n\nRP/0/0/CPU0:xrv1(config-if)#  no shutdown\n\nRP/0/0/CPU0:xrv1(config-if)#\n\nRP/0/0/CPU0:xrv1(config-if)#interface Gi0/0/0/2\n\nRP/0/0/CPU0:xrv1(config-if)#  ipv4 address 99.1.2.9/30\n\nRP/0/0/CPU0:xrv1(config-if)#  no shutdown\n\nRP/0/0/CPU0:xrv1(config-if)#\n\nRP/0/0/CPU0:xrv1(config-if)#interface Gi0/0/0/3\n\nRP/0/0/CPU0:xrv1(config-if)#  ipv4 address 99.1.2.13/30\n\nRP/0/0/CPU0:xrv1(config-if)#  no shutdown\n\nRP/0/0/CPU0:xrv1(config-if)#\n\nRP/0/0/CPU0:xrv1(config-if)#interface Gi0/0/0/4\n\nRP/0/0/CPU0:xrv1(config-if)#  ipv4 address 99.1.2.17/30\n\nRP/0/0/CPU0:xrv1(config-if)#  no shutdown\n\n

In [24]:
print(net_connect.send_command("show ip int b"))


Fri Aug 10 03:02:33.986 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback0                      99.1.0.1        Up              Up       default 
MgmtEth0/0/CPU0/0              172.20.3.101    Up              Up       default 
GigabitEthernet0/0/0/0         99.1.2.1        Up              Up       default 
GigabitEthernet0/0/0/1         99.1.2.5        Up              Up       default 
GigabitEthernet0/0/0/2         99.1.2.9        Up              Up       default 
GigabitEthernet0/0/0/3         99.1.2.13       Up              Up       default 
GigabitEthernet0/0/0/4         99.1.2.17       Up              Up       default 
GigabitEthernet0/0/0/5         99.1.2.21       Up              Up       default 
GigabitEthernet0/0/0/6         99.1.2.25       Up              Up       default 
GigabitEthernet0/0/0/7         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/8         unassigned      Shutdown        Down     default

## 1.4 修改设备其他配置

- isis
- bgp
- segment routing


**下面的代码只是生成的配置文件。并没有配置到设备上。 需要修改部分的代码，最后下发到设备上。


#### 1.4.1 ISIS

In [25]:
isis_template = env.get_template("xrv_isis.j2")

In [27]:
xrv1_isis_data = yaml.load(open("./data/isis/xrv1_isis.yaml").read())

In [28]:
xrv1_isis_data

{'interfaces': [{'name': 'Loopback0'},
  {'name': 'GigabitEthernet0/0/0/0'},
  {'name': 'GigabitEthernet0/0/0/1'},
  {'name': 'GigabitEthernet0/0/0/2'},
  {'name': 'GigabitEthernet0/0/0/3'},
  {'name': 'GigabitEthernet0/0/0/4'},
  {'name': 'GigabitEthernet0/0/0/5'},
  {'name': 'GigabitEthernet0/0/0/6'}],
 'iso_address': '49.0001.0000.0000.0001.00'}

In [29]:
print(isis_template.render(xrv1_isis_data))

router isis lab
 is-type level-2-only
 net 49.0001.0000.0000.0001.00
 nsr
 distribute link-state
 nsf ietf
 log adjacency changes
 address-family ipv4 unicast
  metric-style wide level 2



interface Loopback0
  address-family ipv4 unicast



interface GigabitEthernet0/0/0/0
  point-to-point
  hello-padding disable
  address-family ipv4 unicast



interface GigabitEthernet0/0/0/1
  point-to-point
  hello-padding disable
  address-family ipv4 unicast



interface GigabitEthernet0/0/0/2
  point-to-point
  hello-padding disable
  address-family ipv4 unicast



interface GigabitEthernet0/0/0/3
  point-to-point
  hello-padding disable
  address-family ipv4 unicast



interface GigabitEthernet0/0/0/4
  point-to-point
  hello-padding disable
  address-family ipv4 unicast



interface GigabitEthernet0/0/0/5
  point-to-point
  hello-padding disable
  address-family ipv4 unicast



interface GigabitEthernet0/0/0/6
  point-to-point
  hello-padding disable
  address-family ipv4 unicast




#### 1.4.2 bgp

In [30]:
bgp_template = env.get_template("xrv_bgp.j2")

In [37]:
bgp_datas = []
for x in range(1, 16):
    bgp_datas.append(yaml.load(open("./data/bgp/xrv%s_bgp.yaml" %x).read()))
    


In [39]:
for bgp_data in bgp_datas:
    
    print(bgp_template.render(bgp_data))
    print("*"*10)

router bgp 100
 address-family ipv4 unicast
 !
 address-family vpnv4
 !
 address-family ipv4 sr-policy


 neighbor 99.1.0.2
  remote-as 100
  update-source Loopback0
  address-family ipv4 unicast
  
    route-reflector-client
  
  address-family vpnv4
  
    route-reflector-client
  
  address-family ipv4 sr-policy
  
    route-reflector-client
  


 neighbor 99.1.0.3
  remote-as 100
  update-source Loopback0
  address-family ipv4 unicast
  
    route-reflector-client
  
  address-family vpnv4
  
    route-reflector-client
  
  address-family ipv4 sr-policy
  
    route-reflector-client
  


 neighbor 99.1.0.4
  remote-as 100
  update-source Loopback0
  address-family ipv4 unicast
  
    route-reflector-client
  
  address-family vpnv4
  
    route-reflector-client
  
  address-family ipv4 sr-policy
  
    route-reflector-client
  


 neighbor 99.1.0.5
  remote-as 100
  update-source Loopback0
  address-family ipv4 unicast
  
    route-reflector-client
  
  address-family vpnv4
  
    